Leemos los datos del csv

In [1]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

In [2]:
import pandas as pd
df = pd.read_csv('data/desaparecidos.csv',header=0,names=['estado','municipio','cvegeo','year','total','tasa'], encoding='utf-8')

Agrupamos y quitamos los índices de las columnas estado y year para poder usarlas en el pivot:

In [3]:
df.groupby(['estado','year']).sum().reset_index('year').reset_index('estado').head()

,estado,year,cvegeo,total,tasa
0,Aguascalientes,2006,11066,0,0.000000
1,Aguascalientes,2007,11066,25,24.059914
2,Aguascalientes,2008,11066,7,3.226498
3,Aguascalientes,2009,11066,20,3.495144
4,Aguascalientes,2010,11066,18,7.733581


Creamos la pivot table para tener los años en columnas:

In [4]:
tmp = df.groupby(['estado','year']).sum().reset_index('year').reset_index('estado')
estado_tiempo = tmp.pivot(index='estado',columns='year',values='total')
estado_tiempo.tail()

year,2006,2007,2008,2009,2010,2011,2012,2013,2014
estado,,,,,,,,,
Tamaulipas,15,76,66,207,901,1227,1071,761,831
Tlaxcala,0,0,0,2,0,1,2,6,3
Veracruz de Ignacio de la Llave,2,3,2,1,22,59,45,115,211
Yucatán,0,1,1,2,4,6,10,30,29
Zacatecas,0,1,6,24,30,34,16,46,32


En la maniobra perdimos las claves cvegeo que luego vamos a necesitar para unirlo a las geometrías. Recuperémoslas:

In [5]:
cve_estado = df[['estado','cvegeo']].groupby('estado').first()
cve_estado.tail()

,cvegeo
estado,
Tamaulipas,28001
Tlaxcala,29001
Veracruz de Ignacio de la Llave,30001
Yucatán,31001
Zacatecas,32001


Además tenemos que reindexar las tablas para poder usar los valores de las columnas 'estado' como llaves en la unión:

In [6]:
cve_estado = cve_estado.reset_index('estado')
estado_tiempo = estado_tiempo.reset_index('estado')

Ahora si unimos:

In [7]:
estado_tiempo = pd.merge(cve_estado,estado_tiempo, on='estado')
estado_tiempo.head()

,estado,cvegeo,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,Aguascalientes,1001,0,25,7,20,18,30,14,15,62
1,Baja California,2001,0,7,25,11,8,19,128,177,381
2,Baja California Sur,3001,0,0,2,1,3,2,3,8,3
3,Campeche,4001,0,0,5,0,1,0,0,10,59
4,Chiapas,7001,0,0,0,5,6,9,12,31,41


Ahora leemos el shapefile con las geometrías de los estados:

In [8]:
from geopandas import GeoDataFrame
gf = GeoDataFrame.from_file('data/estados.shp', encoding='utf-8')
gf.columns

Index([u'CVEGEO', u'NOMBRE', u'POB1', u'geometry'], dtype='object')

Trístemente las cvegeo no coinciden, tendremos que intentar unir por nombre. 
Nos quedamos con las columnas necesarias del shape y renombramos la columna nombre para usarla como llave común:

In [9]:
gf = gf[['NOMBRE','POB1','geometry']]
gf = gf.rename(columns = {'NOMBRE':'estado'})
gf.columns

Index([u'estado', u'POB1', u'geometry'], dtype='object')

Unimos los dos conjuntos de datos y lo casteamos a GeoDataFrame:

In [12]:
des_estado = pd.merge(gf,estado_tiempo, on='estado')
des_estado = GeoDataFrame(des_estado)
des_estado.head()

,estado,POB1,geometry,cvegeo,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,Aguascalientes,1184996,POLYGON ((-102.2878651817759 22.41649003941765...,1001,0,25,7,20,18,30,14,15,62
1,Baja California,3155070,(POLYGON ((-115.2104851485454 28.3722493563768...,2001,0,7,25,11,8,19,128,177,381
2,Baja California Sur,637026,(POLYGON ((-109.8006324469839 24.1492608586424...,3001,0,0,2,1,3,2,3,8,3
3,Campeche,822441,POLYGON ((-90.37935699678101 20.84832728853007...,4001,0,0,5,0,1,0,0,10,59
4,Coahuila de Zaragoza,2748391,POLYGON ((-102.3107926469074 29.87694857356086...,5001,1,55,102,117,227,258,136,233,103


Antes de guardar el resultado como shape necesitamos renombrar las columnas de los años porque son enteros y no strings.



In [14]:
des_estado = des_estado.rename(columns={2006:'2006',2007:'2007',2008:'2008',2009:'2009',2010:'2010',2011:'2011',2012:'2012',2013:'2013',2014:'2014'})


Guardamos el resultado como shapefile para leerlo con clusterpy

In [15]:
des_estado.to_file('data/desaparecidos_estatal',encoding='utf-8')

In [18]:
for c in des_estado.columns:
    print des_estado[c].dtype
    